# Convert Laz to DEM using CloudCompare

This is done in three steps:

## 1. Create working directories
4 batches of 50 laz files (ex: 0-49, 50-99, 100-149, 150-199) will be downloaed into a folder that can contain 200 files (ex: laz_utm_0-199).

Create folders to hold downladed files. Example: E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_0-199. This can be done by running the first cell of 1_1_Maryland_download_laz_tiles_use_CloudCompare_to_convert_to_DEM

## 2. Download the files

Download the files from Google Drive, unzip into directory. Example: /content/drive/MyDrive/crane_maryland/laz/laz_fema_2012_forested_utm/batch_50/laz_utm_0-49




In [7]:
# 1. Create working directories
# Run this only once
# set up working directories to hold downloaded files
import os
input_laz_utm_fp_base = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\"

chunk_amount = 200
total_files = 2154

for count in range(0, total_files , 200):

    new_dir_file_path = os.path.join(input_laz_utm_fp_base, ("laz_utm_"+str(count)+"-"+str(count+chunk_amount-1)))
    print(new_dir_file_path)
    if not (os.path.exists(new_dir_file_path)):  
        print("Does not exist.")
        os.mkdir(new_dir_file_path)
    else:
        print("Exists.")

E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_0-199
Exists.
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_200-399
Exists.
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_400-599
Exists.
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_600-799
Exists.
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_800-999
Exists.
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1000-1199
Exists.
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1200-1399
Exists.
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1400-1599
Exists.
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1600-1799
Exists.
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_1800-1999
Exists.
E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\laz_utm_2000-2199
Exists.


## 2. Download the files

Manual download

## 3. Convert Laz to Raster with CloudCompare

+ update the directory structure
+ Run the process below to perform a batch conversion

In [ ]:
# E:\Program Files\CloudCompare>cloudcompare -O 
#'E:/a_new_orgs/carleton/hist5706-maryland/lidar_files/reprojected/20120323_001415_r_utm.laz' 
#-SET_ACTIVE_SF 6 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1

import subprocess
import time

# example command
# C:\LAStools\bin\las2las -i "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\20120323_001460.laz" -target_utm 18north -o "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\20120323_001460_r_utm.laz"

input_laz_utm_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_0-199\\"
#output_raster_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\rasters\\"


import os
#process .laz files one at a time
for root, dirs, files in os.walk(input_laz_utm_fp, topdown=False):
   for name in files:      
      #print(name)
      name_ext = name[-4:].lower()
      if(name_ext==".laz"):
          laz_in_path = os.path.join(input_laz_utm_fp, name)
          #raster_out_path = os.path.join(output_raster_fp, (name[:-4]+".tif"))
          command = '"E:\Program Files\CloudCompare\cloudcompare" -SILENT -O ' + laz_in_path + " -SET_ACTIVE_SF 8 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z "
          print("!"+command)
          subprocess.call(command, shell=True)
          # just in case, let the sub process run
          time.sleep(4)
        

"""
Rasterizes the loaded clouds by projecting them inside a regular grid.
Optional settings are:

-VERT_DIR {0/1/2} to specify the projection dimension (0=X / 1=Y / 2=Z) - default is 2 (Z)
-PROJ {MIN/AVG/MAX} to specify how the altitude of each cell is computed (minimum, average or maximum altitude of the projected points). Default is AVG (average)
-SF_PROJ {MIN/AVG/MAX} to specify how the SF value(s) of each cell are computed (minimum, average or maximum altitude of the projected points SF value(s)). Default is AVG (average)
-EMPTY_FILL {MIN_H/MAX_H/CUSTOM_H/INTERP} to specify how empty cells should be filled (MIN_H = min height / MAX_H = max height / CUSTOM_H = custom value - see CUSTOM_HEIGHT option below / INTERP = interpolate). Default behavior is 'leave cells empty'
-CUSTOM_HEIGHT {value} to define the custom height filling value (if the 'CUSTOM_H' strategy is used (see above)
-OUTPUT_CLOUD to output the result as a cloud (this is the default option if no other output format is defined)
-OUTPUT_MESH to output the result as a mesh
-OUTPUT_RASTER_Z to output the result as a geotiff raster (only altitudes, no RGB)
-OUTPUT_RASTER_Z_AND_SF to output the result as a geotiff raster (altitudes + all SFs by default, no RGB)
-OUTPUT_RASTER_RGB to output the result as a geotiff raster (RGB)        
"""

In [11]:
!"e:\Program Files\CloudCompare\cloudcompare" -O E:\a_new_orgs\carleton\\hist5706-maryland\\lidar_files\\reprojected\\20120203_17SPD7586_utm.laz-SET_ACTIVE_SF 6 -FILTER_SF 1.1 2.1 -RASTERIZE -GRID_STEP 1 -EMPTY_FILL INTERP -OUTPUT_RASTER_Z E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\rasters\\20120203_17SPD7586_utm.tif

'e:\\Program' is not recognized as an internal or external command,
operable program or batch file.


# Move tifs

In [ ]:
import os
import shutil
# Divide files into groups of 200 so they can be downloaded and processed in managably sized chunks.
file_counter = 0

dem_tif_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_0-49\\"
dem_tif_fp_dest = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\laz_utm_0-49\\dem_tifs\\"

for file_name in os.listdir(dem_tif_fp):
    origin_file_path = os.path.join(dem_tif_fp,file_name)
    print(origin_file_path)
    if(os.path.isfile(origin_file_path)):  
        if(origin_file_path[-4:]==".tif"):
            shutil.copy(origin_file_path, os.path.join(dem_tif_fp_dest,(file_name[:22]+".tif")))
            print(file_counter, origin_file_path)
            file_counter = file_counter + 1        

In [5]:
fn = "20120203_17SQD2798_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-11-08_18h04_32_409.tif"
print(fn[:22])

20120203_17SQD2798_utm


# Set crs of raster - Not yet working - Ignore below

In [19]:
import subprocess
import time
import rasterio

# example command
# C:\LAStools\bin\las2las -i "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\20120323_001460.laz" -target_utm 18north -o "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\20120323_001460_r_utm.laz"


input_raster_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\"
output_raster_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\rasters\\"


# Set crs
import os
#process .tif files one at a time
for root, dirs, files in os.walk(input_raster_fp, topdown=False):
   for name in files:      
      #print(name)
      name_ext = name[-4:].lower()
      if(name_ext==".tif"):
          tif_out_path = os.path.join(input_raster_fp, name)
          tif_in_path = os.path.join(output_raster_fp, (name[:19]+"26918"+name_ext))
          with rasterio.open(tif_in_path, "w", crs='EPSG:26918') as dest:
              dest.write(out_image)         
          #command = 'gdal_edit.bat -a_srs EPSG:26985 "' + tif_in_path + '"'
                    
          #print("!"+command)
          #subprocess.call(command, shell=True)
          # just in case, let the sub process run
          time.sleep(4)            

ModuleNotFoundError: No module named 'rasterio'

In [20]:
!gdal_edit.bat -a_srs EPSG:26985 "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125.tif"

'gdal_edit.bat' is not recognized as an internal or external command,
operable program or batch file.


# Make slope file - Not yet working

In [16]:
import subprocess
import time

# example command
# gdaldem slope E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125.tif 
# C:/Users/User/AppData/Local/Temp/processing_QIEzpg/6c1743fc49544eba9ae8c25d891f86e8/OUTPUT.tif
# -of GTiff -b 1 -s 1.0


input_raster_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\"
output_slope_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\slope\\"


# Set crs
import os
#process .tif files one at a time
for root, dirs, files in os.walk(input_raster_fp, topdown=False):
   for name in files:      
      #print(name)
      name_ext = name[-4:].lower()
      if(name_ext==".tif"):
          tif_in_path = os.path.join(input_raster_fp, name)
          slope_out_path = os.path.join(output_slope_fp, (name[:-4]+"_slope"+name_ext))
        
          #command = '"C:\\Program Files\\QGIS 3.16\\bin\\gdaldem.exe" slope "' + tif_in_path + '" "' + slope_out_path + " -of GTiff -b 1 -s 1.0"
          command = '"E:\\Program Files\\QGIS 3.10\\bin\\gdaldem" slope "' + tif_in_path + '" "' + slope_out_path + '" -of GTiff -b 1 -s 1.0'
            
            
                    
          print("!"+command)
          subprocess.call(command, shell=True)
          # just in case, let the sub process run
          time.sleep(4)  

!"E:\Program Files\QGIS 3.10\bin\gdaldem" slope "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\reprojected\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125.tif" "E:\a_new_orgs\carleton\hist5706-maryland\lidar_files\slope\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125_slope.tif" -of GTiff -b 1 -s 1.0


In [11]:
!"E:\\Program Files\\QGIS 3.10\\bin\\gdaldem" slope "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125.tif" "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\slope\\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125_slope.tif" -of GTiff -b 1 -s 1.0


0...10...20...30...40...50...60...70...80...90...100 - done.


In [15]:
!"E:\\Program Files\\QGIS 3.10\\bin\\gdaldem" slope "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\reprojected\\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125.tif" "E:\\a_new_orgs\\carleton\\hist5706-maryland\\lidar_files\\slope\\20120203_17SPD7277_utm_FILTERED_[1.1_2.1]_RASTER_Z_2021-10-30_16h50_37_125_slope.tif" -of GTiff -b 1 -s 1.0

0...10...20...30...40...50...60...70...80...90...100 - done.


In [1]:
import os
import geopandas as gpd

lidar_tiles_forested_polys_fp = "E:\\a_new_orgs\\carleton\\hist5706-maryland\\hist5706-maryland\\fema_forested\\2012_fema_md_forested.shp"

"""
# Create an empty geopandas GeoDataFrame for duplicates
lidar_tiles_forested_polys_df = gpd.GeoDataFrame()
lidar_tiles_forested_polys_df['Index'] = None
lidar_tiles_forested_polys_df['Name'] = None
lidar_tiles_forested_polys_df['URL'] = None
lidar_tiles_forested_polys_df['geometry'] = None
"""

lidar_tiles_forested_polys_df.crs = ('EPSG:26985')

if os.path.exists(lidar_tiles_forested_polys_fp):
    lidar_tiles_forested_polys = gpd.read_file(lidar_tiles_forested_polys_fp)
    
    print("crs", lidar_tiles_forested_polys.crs)
    print(type(lidar_tiles_forested_polys))

    print("lidar_tiles_forested_polys",lidar_tiles_forested_polys.shape)
    # crs for Maryland
    area_crs = 26985
    land_use_polys.to_crs(area_crs)

    for index, row in lidar_tiles_forested_polys.iterrows():
       #print("row",row)

       print(row[1],row[2],row[3])

ModuleNotFoundError: No module named 'geopandas'